#### 개요

학습용 데이터와 온라인 시험용 테스트 데이터를 준비하고, 제약조건(전주/전선 수 10개 미만 제외)을 검사해 레코드를 제거하고, 학습에 필요한 컬럼만 추려 데이터를 저장함
* 배치/온라인 부분이 구분되어 구현됨

In [1]:
import pandas as pd
from io import StringIO

import aidd.sys.config as cfg
from aidd.utils.data_io import get_provide_data, save_data

#### 학습대상 레코드와 컬럼 추출

##### 공통함수 생성

In [2]:
def preparation_data(pdf_dict=None, is_batch=True):
    # 제약조건을 충족한 학습용 레코드와 컬럼이 저장된 딕셔너리
    df_dict = {}
    
    # 공사비 데이터 학습대상 레코드 추출(제약조건이 있어 별도 처리)
    key = cfg.DATA_SETs[0]
    df = pdf_dict[key]
    # (전주/전선 수를 제외한) 공사비 데이터 부분에서 학습 대상 레코드 조건
    # * 접수종류명(ACC_TYPE_NAME), 계약전력(CONT_CAP),
    # * 공사형태코드(CONS_TYPE_CD), 총공사비(TOTAL_CONS_COST)
    modeling_records = \
        (df.ACC_TYPE_NAME  == cfg.CONSTRAINTS['ACC_TYPE_NAME']) & \
        (df.CONT_CAP        < cfg.CONSTRAINTS['MAX_CONT_CAP']) & \
        (df.CONS_TYPE_CD   == cfg.CONSTRAINTS['CONS_TYPE_CD']) & \
        (df.TOTAL_CONS_COST < cfg.CONSTRAINTS['MAX_TOTAL_CONS_COST'])
    df = df[modeling_records].reset_index(drop=True)
    cons_df = df[cfg.COLS['PP'][key]['SOURCE']]
    df_dict[key] = cons_df
    
    # 전주/전선/인입선 데이터 제약조건 처리
    for key in cfg.DATA_SETs[1:]:
        df = pdf_dict[key]
        df = df[df.CONS_ID.isin(cons_df.CONS_ID)]
        df_dict[key] = df[cfg.COLS['PP'][key]['SOURCE']]
        
    # 데이터 저장
    if is_batch:
        for key in cfg.DATA_SETs:
            save_data(df_dict[key], f'MERGE,BATCH,{key}')
    else:
        # 이 부분을 서비스 시 수행할 필요 없으니 주석처리
        for key in cfg.DATA_SETs:
            save_data(df_dict[key], f'MERGE,ONLINE,{key}')
            
    return df_dict

##### 배치 작업

In [3]:
# 제공받은 데이터가 저장된 딕셔너리
pdf_dict = get_provide_data()

Data Type: CONS, Size: (19052, 143), pTime: 0:00:27.832838
Data Type: POLE, Size: (38533, 63), pTime: 0:00:24.872879
Data Type: LINE, Size: (40019, 77), pTime: 0:00:31.416517
Data Type: SL, Size: (22632, 57), pTime: 0:00:12.980993


In [4]:
batch_df_dict = preparation_data(pdf_dict=pdf_dict)

In [5]:
batch_df_dict['CONS'].max()

CONS_ID                   70HC20234512
TOTAL_CONS_COST               29998731
CONS_TYPE_CD                         2
LAST_MOD_DATE      2023-09-01 15:54:47
LAST_MOD_EID                MMP1900092
OFFICE_NAME                       충주지사
CONT_CAP                            49
ACC_TYPE_NAME                신설(상용/임시)
dtype: object

In [6]:
# 제약조건을 충족한 데이터 크기
for key in cfg.DATA_SETs:
    print(f'Data Type: {key}, Size: {batch_df_dict[key].shape}')

Data Type: CONS, Size: (15335, 8)
Data Type: POLE, Size: (27896, 6)
Data Type: LINE, Size: (30348, 11)
Data Type: SL, Size: (17034, 6)


##### 온라인 작업

In [7]:
def get_online_sample(pdf_dict=None):
    """온라인 서비스 샘플 데이터 생성 함수
    * 온라인 서비스나 온라인 서비스를 위한 전처리 프로세스 테스트를 위한
      샘플 데이터를 생성하는 함수로, 대상 데이터는 `config`에 지정되어 있음
    * 실제 온라인 작업에선 사용하지 않음
    Args:
        data (dict): 학습용으로 제공받은 전체 데이터
            - `key: df`로 구성된 딕셔너리로 제공됨
    Returns:
        jsons (dict): Online으로 받을 예정인 JSON 데이터의 딕셔너리
    """
    # 전체 데이터에서 샘플 데이터만 추출한 데이터프레임 딕셔너리
    jsons = {}
    for key in cfg.DATA_SETs:
        df = pdf_dict[key]
        df = df[df.CONS_ID.isin(cfg.CHECK_CONS_IDS)]
        modeling_cols = cfg.COLS['PP'][key]['SOURCE']
        # 데이터프레임을 이용해 날자 데이터를 JSON으로 변경해서 다시 복원하기 위해서는,
        # JSON 데이터로 만들기 전에 숫자로 만들고, 복원 시 이 숫자를 이용해 날자를 만들어야 함
        # 이 부분을 수행하지 않으면 JSON변환 과정에서 데이터가 짤리면서 1970년대 데이터로 변환됨
        if key == 'CONS':
            df.loc[:, 'LAST_MOD_DATE'] = df.LAST_MOD_DATE.astype(int)
        jsons[key] = df[modeling_cols].to_json(orient='records')
        
    return jsons

In [8]:
# RESTful 데이터 받기(임의 생성)
js_dict = get_online_sample(pdf_dict=pdf_dict)

###### 수정 예정
이 아래 부분은 추후 수정할 사항이 많아 보임(RESTful JSON 데이터를 데이터프레임으로 변환)

In [9]:
# Web데이터를 데이터프레임으로 전환
js_df_dict = {}
for key in js_dict.keys():
    js_df_dict[key] = pd.read_json(StringIO(js_dict[key]))

In [10]:
# 총공사비가 없는 경우에는 총공사비 컬럼을 0으로 추가
# 예측해야할 값이긴 하지만 온라인/배치 전처리를 같은 함수로 하기 위해 
# 컬럼만 추가함
if 'TOTAL_CONS_COST' not in js_df_dict['CONS'].keys():
    js_df_dict['CONS'].loc[:, 'TOTAL_CONS_COST'] = 0

In [11]:
# 날짜형 데이터 복원(JSON변환 과정에서 숫자로 처리해 둠)
js_df_dict['CONS'].LAST_MOD_DATE = \
    pd.to_datetime(js_df_dict['CONS'].LAST_MOD_DATE)

In [12]:
# 온라인으로 들어온 데이터의 경우 CONS_ID가 정수형으로 들어오는데,
# 배치 작업과 병행처리를 위해 문자형으로 변환
for key in cfg.DATA_SETs:
    df = js_df_dict[key]
    df.CONS_ID = df.CONS_ID.astype(str)
    js_df_dict[key] = df

In [13]:
js_df_dict['CONS'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CONS_ID          3 non-null      object        
 1   TOTAL_CONS_COST  3 non-null      int64         
 2   CONS_TYPE_CD     3 non-null      int64         
 3   LAST_MOD_DATE    3 non-null      datetime64[ns]
 4   LAST_MOD_EID     3 non-null      object        
 5   OFFICE_NAME      3 non-null      object        
 6   CONT_CAP         3 non-null      int64         
 7   ACC_TYPE_NAME    3 non-null      object        
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 320.0+ bytes


In [14]:
# 온라인용 데이터 딕셔너리
online_df_dict = preparation_data(pdf_dict=js_df_dict, is_batch=False)

In [15]:
# 제약조건을 충족한 데이터 크기
for key in cfg.DATA_SETs:
    print(f'Data Type: {key}, Size: {online_df_dict[key].shape}')

Data Type: CONS, Size: (3, 8)
Data Type: POLE, Size: (9, 6)
Data Type: LINE, Size: (9, 11)
Data Type: SL, Size: (3, 6)


In [16]:
online_df_dict['CONS'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CONS_ID          3 non-null      object        
 1   TOTAL_CONS_COST  3 non-null      int64         
 2   CONS_TYPE_CD     3 non-null      int64         
 3   LAST_MOD_DATE    3 non-null      datetime64[ns]
 4   LAST_MOD_EID     3 non-null      object        
 5   OFFICE_NAME      3 non-null      object        
 6   CONT_CAP         3 non-null      int64         
 7   ACC_TYPE_NAME    3 non-null      object        
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 320.0+ bytes
